In [1]:
import pandas as pd
import pandas_datareader as pdr

import requests_cache
session = requests_cache.CachedSession(cache_name='stocks-cache', backend='sqlite')

In [2]:
symbols = ['CL=F', 'BTC-USD', 'RUB=X', 'GC=F', '^GSPC', '^IXIC']

ticks = [
    pdr.DataReader(
        sym,
        start='2014-01-01',
        end='2020-05-01',
        data_source='yahoo',
        session=session
    ).Close.fillna(method='ffill').rename(sym)
    for sym in symbols
]

ticks = [t[~t.index.duplicated()] for t in ticks]

ticks_df = pd.concat(ticks, join='inner', axis=1)
ticks_df = ticks_df[~(ticks_df < 1).any(axis=1)]

ticks_df.corr().style.background_gradient(cmap='bwr', vmin=-1, vmax=1)

,CL=F,BTC-USD,RUB=X,GC=F,^GSPC,^IXIC
CL=F,1.000000,0.299952,-0.666507,-0.073561,0.312550,0.251984
BTC-USD,0.299952,1.000000,0.065245,0.609711,0.838919,0.833857
RUB=X,-0.666507,0.065245,1.000000,0.179455,0.114210,0.178153
GC=F,-0.073561,0.609711,0.179455,1.000000,0.692176,0.714099
^GSPC,0.312550,0.838919,0.114210,0.692176,1.000000,0.991147
^IXIC,0.251984,0.833857,0.178153,0.714099,0.991147,1.000000


In [3]:
change_df = ticks_df.pct_change().dropna()
change_df.corr().style.background_gradient(cmap='bwr', vmin=-1, vmax=1)

,CL=F,BTC-USD,RUB=X,GC=F,^GSPC,^IXIC
CL=F,1.000000,0.041884,-0.137530,0.106187,0.286956,0.266799
BTC-USD,0.041884,1.000000,-0.029105,0.038291,0.109061,0.102011
RUB=X,-0.137530,-0.029105,1.000000,-0.058473,-0.131074,-0.127754
GC=F,0.106187,0.038291,-0.058473,1.000000,-0.009778,-0.021037
^GSPC,0.286956,0.109061,-0.131074,-0.009778,1.000000,0.959762
^IXIC,0.266799,0.102011,-0.127754,-0.021037,0.959762,1.000000


In [48]:
change_df = ticks_df.rolling(5).sum().dropna().pct_change().dropna()
change_df.corr().style.background_gradient(cmap='bwr', vmin=-1, vmax=1)

,CL=F,BTC-USD,RUB=X,GC=F,^GSPC,^IXIC
CL=F,1.000000,0.086011,-0.413454,0.080145,0.354026,0.296728
BTC-USD,0.086011,1.000000,-0.080126,0.062810,0.177097,0.181907
RUB=X,-0.413454,-0.080126,1.000000,-0.099211,-0.351818,-0.315893
GC=F,0.080145,0.062810,-0.099211,1.000000,0.043839,0.014652
^GSPC,0.354026,0.177097,-0.351818,0.043839,1.000000,0.954748
^IXIC,0.296728,0.181907,-0.315893,0.014652,0.954748,1.000000


In [50]:
change_df = ticks_df[ticks_df.index < '2017-01-01'].rolling(5).sum().pct_change().dropna()
change_df.corr().style.background_gradient(cmap='bwr', vmin=-1, vmax=1)

,CL=F,BTC-USD,RUB=X,GC=F,^GSPC,^IXIC
CL=F,1.000000,0.035747,-0.466252,-0.074728,0.387988,0.344987
BTC-USD,0.035747,1.000000,-0.129088,-0.086469,0.061517,0.071176
RUB=X,-0.466252,-0.129088,1.000000,-0.029590,-0.353168,-0.308580
GC=F,-0.074728,-0.086469,-0.029590,1.000000,-0.205004,-0.260066
^GSPC,0.387988,0.061517,-0.353168,-0.205004,1.000000,0.956612
^IXIC,0.344987,0.071176,-0.308580,-0.260066,0.956612,1.000000


In [5]:
walcl = pdr.DataReader(
    'WALCL',
    start='2003-01-01',
    end='2020-05-01',
    data_source='fred',
    session=session
)

snp = pdr.DataReader(
    '^GSPC',
    start='2003-01-01',
    end='2020-05-01',
    data_source='yahoo',
    session=session
).Close.rename('^GSPC')

stat = pd.concat([walcl, snp], join='inner', axis=1)

In [21]:
stat.corr().style.background_gradient(cmap='bwr', vmin=-1, vmax=1)

,WALCL,^GSPC
WALCL,1.000000,0.778495
^GSPC,0.778495,1.000000
